# 共起語分析スクリプト(Simple)

このスクリプトは、spaCyライブラリを使用して事前に解析されたテキストデータに対して共起語分析を行う。文単位での分析、共起範囲を指定した分析、正規表現による柔軟な検索などの機能を含む。

## 使用方法

### 解析データの読み込み

1. 解析済みのテキストデータ（.spacyファイル）とそれに対応するテキストファイル（.txt）を用意する。
2. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `main_directory`: 親ディレクトリへの相対パス  
 - `subdirectories`: 読み込む子ディレクトリ名のリスト 
 - `nlp_model`: 解析に利用したモデル名
 - `selected_file_names`: 特定のファイルのみを読み込む場合はここにtxtファイル名を追加（空リストの場合は全ファイルを読み込む）

### 共起語分析（文単位）の実行

1. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `search_type`: 検索モード（word, lemma, phrase, regex）
 - `keyword`: 検索するキーワード
 - `part_of_speech`: キーワードの品詞（word, lemma検索のみ）
 - `output_pos`: フィルターする品詞のリスト
 - `max_output`: 出力する行数
 - `output_file`: 出力するCSVファイル名
2. スクリプトを実行すると、指定したファイル名でCSVファイルが出力される。

### 例文検索機能の実行

1. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `lemma_to_extract`: 抽出する単語の原形
 - `pos_to_extract`: 抽出する単語の品詞
 - `output_text_file`: 抽出した文を保存するテキストファイル名
2. スクリプトを実行すると、指定したファイル名で例文がテキストファイルに出力される。

## 分析内容

### 1. 共起語分析（文単位） 

文単位でキーワードを検索し、文内の共起語を分析する。検索モードには単語、原形、フレーズ、正規表現が選択できる。結果はCSVファイルとして出力される。

### 2. 共起語分析（共起範囲指定）

キーワードの前後の共起範囲を指定して、その範囲内の共起語を分析する。検索モードには単語、原形、フレーズ、正規表現が選択できる。結果はCSVファイルとして出力される。

### 3. 例文検索機能

共起語分析の結果から、指定した単語と品詞に対応する例文を抽出し、テキストファイルに出力する。

# **ライブラリのインポート(最初に実行)**

In [1]:
import spacy
import os
import pandas as pd
import re
from spacy.tokens import DocBin
from spacy.matcher import PhraseMatcher
from collections import defaultdict

# **解析データの読み込み**
## ユーザーが設定するパラメーター

In [2]:
# 基本設定
main_directory = "processed_data"  # 親ディレクトリへの相対パス
subdirectories = ["sub1"]  # 読み込む子ディレクトリ名のリスト
nlp_model = "en_core_web_sm"  # 解析に利用したモデル名

# 特定のファイルのみを読み込む場合は以下のリストにtxtファイル名を追加(空リストの場合は全ファイルを読み込む)
selected_file_names = []

## 解析データ読み込みの実行

In [ ]:
# spaCyの言語モデルをロード
nlp = spacy.load(nlp_model)

# 解析結果を格納する辞書の初期化
docs_dict = {}
# フィルタリングされたDocオブジェクトを格納する辞書の初期化
filtered_docs = {}

# 指定されたサブディレクトリ内のファイルを読み込む
for subdir in subdirectories:
    directory = os.path.join(main_directory, subdir)
    
    # サブディレクトリ内のファイルを読み込む
    for filename in os.listdir(directory): 
        if filename.endswith(".spacy"):  # .spacyファイルならば
            spacy_path = os.path.join(directory, filename)
            txt_filename = filename.replace(".spacy", ".txt")  # .spaCyファイルに対応するテキストファイル名を作成
            txt_path = os.path.join(directory, txt_filename)  # .spacyファイルに対応するテキストファイルのパス
            
            # .spacyファイルからDocオブジェクトを読み込む
            doc_bin = DocBin().from_disk(spacy_path)  # DocBinオブジェクトを読み込む
            docs = list(doc_bin.get_docs(nlp.vocab))  # Docオブジェクトをリストに格納
            
            # 対応する.txtファイルからファイル名を読み込む
            if os.path.exists(txt_path):
                with open(txt_path, 'r', encoding='utf-8') as f:
                    file_names = [line.strip() for line in f.readlines()]  # ファイル名をリストに格納
                
                # Docオブジェクトとファイル名の対応を辞書に格納
                for doc, fname in zip(docs, file_names):
                    docs_dict[fname] = doc
            else:
                print(f"警告: {txt_filename} に対応するテキストファイルが存在しません。")

# ユーザーがファイル名を指定した場合、それに対応するDocオブジェクトをフィルタリング
if selected_file_names:
    missing_files = []  # 存在しないファイル名を格納するリスト
    for fname in selected_file_names:
        if fname in docs_dict:  # 指定されたファイル名が読み込んだデータに存在する場合
            filtered_docs[fname] = docs_dict[fname]
        else:
            missing_files.append(fname)
    
    if missing_files:
        print(f"警告: 次の指定されたファイルは存在しません: {', '.join(missing_files)}")
    if filtered_docs:
        print("指定された一部のデータが読み込まれました。")
    else:
        print("指定されたファイルに対応するデータが見つかりませんでした。")
else:
    filtered_docs = docs_dict
    print("全てのデータが読み込まれました。")
print(f"読み込まれたデータ数: {len(filtered_docs)}")
print(f"読み込まれたファイル名: {list(filtered_docs.keys())}")

# **共起語分析（文単位）**
## ユーザーが設定するパラメーター

In [ ]:
# 基本設定
search_type = "phrase" # word, lemma, phrase, regex のいずれかを指定
keyword = "hello world"  # 検索するキーワードを設定

# 追加設定
part_of_speech = "None"  # キーワードの品詞を設定(word, lemma検索のみ)

# 出力設定
output_pos = ["NOUN", "VERB", "ADJ", "PROPN", "ADV"]  # フィルターする品詞をリストで設定
max_output = 150  # 出力する行数を指定
output_file = "sentence_level_collocations.csv"  # 出力するCSVファイルの名前を設定

## 共起語分析の実行

In [ ]:
def search_word(doc, keyword, part_of_speech, collocation_dict, file_name):
    """
    文単位で単語を検索し、共起語を記録する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワード。
        part_of_speech (str): キーワードの品詞。Noneの場合は品詞を指定しない。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
    """
    for sent in doc.sents:
        keyword_found = False
        for token in sent:
            # 品詞が指定されていない場合、または品詞が一致する場合、かつキーワード(小文字)が一致する場合
            if (part_of_speech is None or token.pos_ == part_of_speech) and token.text.lower() == keyword.lower():
                keyword_found = True 
                break # キーワードが見つかったら文の処理を終了
                
        if keyword_found:
            # キーワードにマッチしないトークンのみを抽出してリストに格納
            filtered_tokens = [
                token for token in sent
                if not(
                    (part_of_speech is None or token.pos_ == part_of_speech) and
                    token.text.lower() == keyword.lower()
                )
            ]
            record_collocations(filtered_tokens, collocation_dict, file_name)

def search_lemma(doc, keyword, part_of_speech, collocation_dict, file_name):
    """
    文単位で単語の原形を検索し、共起語を記録する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワードの原形。
        part_of_speech (str): キーワードの品詞。Noneの場合は品詞を指定しない。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
    """
    for sent in doc.sents:
        lemma_found = False
        for token in sent:
            # 品詞が指定されていない場合、または品詞が一致する場合、かつキーワードの基本形(小文字)が一致する場合
            if (part_of_speech is None or token.pos_ == part_of_speech) and token.lemma_.lower() == keyword.lower():
                lemma_found = True
                break
                
        if lemma_found:
            filtered_tokens = [
                token for token in sent
                if not(
                    (part_of_speech is None or token.pos_ == part_of_speech) and
                    token.lemma_.lower() == keyword.lower()
                )
            ]
            record_collocations(filtered_tokens, collocation_dict, file_name)


def search_phrase(doc, phrase, collocation_dict, file_name):
    """
    文単位でフレーズを検索し、共起語を記録する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        phrase (str): 検索するフレーズ。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
    """
    # PhraseMatcherの初期化
    matcher = PhraseMatcher(doc.vocab, attr="LOWER")
    phrase_patterns = [nlp(phrase)]  # フレーズをDocオブジェクトに変換
    matcher.add("PHRASE", phrase_patterns)

    # ドキュメント内でフレーズを検索
    matches = matcher(doc)

    for sent in doc.sents:
        spans = []  # マッチしたフレーズのスパンを記録するリスト

        for match_id, start, end in matches:
            span = doc[start:end]
            if span.sent == sent: # マッチしたフレーズが処理中の文内にある場合, スパンを記録
                spans.append((span.start_char - sent.start_char, span.end_char - sent.start_char))

        # spansリストに含まれないトークンのみを抽出して共起語を記録
        filtered_tokens = [token for token in sent if not any(start <= token.idx - sent.start_char < end for start, end in spans)]
        record_collocations(filtered_tokens, collocation_dict, file_name)

def search_regex(doc, keyword, collocation_dict, file_name):
    """
    文単位で正規表現によるパターンを検索し、共起語を記録する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索する正規表現パターン。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
    """
    for sent in doc.sents:
        spans = [] # この文で見つかったキーワードのindexを記録するリスト

        for match in re.finditer(keyword, sent.text):  # 文内でキーワードにマッチする箇所を探す
            start_idx = match.start()
            end_idx = match.end()
            spans.append((start_idx, end_idx))  # マッチしたキーワードの範囲(開始と終了のインデックス)を記録

        # spansリストに含まれないトークンのみを抽出して共起語を記録
        filtered_tokens = [token for token in sent if not any(start <= token.idx - sent.start_char < end for start, end in spans)]
        record_collocations(filtered_tokens, collocation_dict, file_name)


def record_collocations(tokens, collocation_dict, file_name):
    """
    共起語の情報を辞書に記録する関数。

    Args:
        tokens (list): 共起語のトークンのリスト。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
    """
    # 集計した共起語の情報を辞書に記録
    for token in tokens:
        key = (token.lemma_, token.pos_)
        collocation_dict[key]['lemma'] = token.lemma_
        collocation_dict[key]['pos'] = token.pos_
        collocation_dict[key]['freq'] += 1
        collocation_dict[key]['indexes'].append((file_name, token.i))



def perform_collocation_search(doc, keyword, search_type, collocation_dict, part_of_speech, file_name):
    """
    共起語分析を実行する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワード。
        search_type (str): 検索モード（word, lemma, phrase, regex）。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        part_of_speech (str): キーワードの品詞。Noneの場合は品詞を指定しない。
        file_name (str): 処理中のファイル名。

    Raises:
        ValueError: 無効な検索モードが指定された場合。
    """
    if search_type == 'word':
        search_word(doc, keyword, part_of_speech, collocation_dict, file_name)
    
    elif search_type == 'lemma':
        search_lemma(doc, keyword, part_of_speech, collocation_dict, file_name)
    
    elif search_type == 'phrase':
        search_phrase(doc, keyword, collocation_dict, file_name)
    
    elif search_type == 'regex':
        search_regex(doc, keyword, collocation_dict, file_name)


def save_collocations_to_csv(collocation_dict, output_file, output_pos=None, max_output=None):
    """
    共起語の情報をCSVファイルに保存する。

    Args:
        collocation_dict (defaultdict): 共起語情報を格納する辞書
        output_file (str): 出力ファイル名
        output_pos (list, optional): 出力する品詞のリスト。デフォルトはNone。
        max_output (int, optional): 出力する結果の最大数。デフォルトはNone。
    """
    # 辞書からデータフレームを作成
    df = pd.DataFrame.from_dict(collocation_dict, orient='index')
    
    # 全品詞に対するソート
    df_sorted_all = df.sort_values(by='freq', ascending=False)
    if max_output is not None:
        df_sorted_all = df_sorted_all.head(max_output)
    
    # 指定された品詞でフィルタリング
    dfs_filtered = []
    if output_pos:
        for pos in output_pos:
            df_filtered = df[df['pos'] == pos].sort_values(by='freq', ascending=False)
            if max_output is not None:
                df_filtered = df_filtered.head(max_output)
            df_filtered = df_filtered[['lemma', 'freq']].reset_index(drop=True)
            df_filtered.columns = [f'lemma_{pos}', f'freq_{pos}']
            dfs_filtered.append(df_filtered)
    
    # 指定された品詞に基づくデータフレームを結合
    df_filtered_combined = pd.concat(dfs_filtered, axis=1)
    # 全品詞の共起語データを追加
    df_final = pd.concat([df_filtered_combined, df_sorted_all[['lemma', 'pos', 'freq']].reset_index(drop=True)], axis=1)
    
    # CSVとして保存
    df_final.to_csv(output_file, index=False)


# 共起語を格納するための辞書
collocation_dict = defaultdict(lambda: {'lemma': '', 'pos': '', 'freq': 0, 'indexes': []})

# 検索タイプが無効な場合はエラーを返す
if search_type not in ["word", "lemma", "regex", "phrase"]:
    raise ValueError("検索タイプは 'word', 'lemma', 'regex', 'phrase' のいずれかである必要があります。")
    
# 品詞が指定されていない場合、検索対象の品詞をNoneに設定, search_typeがphraseまたはregexの場合もNoneに設定
if part_of_speech == "NONE" or part_of_speech == "" or search_type == 'phrase' or search_type == 'regex': 
    part_of_speech = None

# フィルタリングされたDocオブジェクトに対して共起語分析を実行
for file_name, doc in filtered_docs.items():
    perform_collocation_search(doc, keyword, search_type, collocation_dict, part_of_speech, file_name)

# 共起語の総数を計算し、結果を表示
total_matches = len(collocation_dict)
if total_matches > 0:
    # コンソールに共起語分析の結果概要を表示
    print(f"共起語分析の結果: {total_matches}件の共起語が見つかりました。")
    print(f"検索モード: {search_type}, キーワード: {keyword}, 品詞: {part_of_speech if part_of_speech else 'None'}")
    
    # 結果をCSVファイルに出力する関数を呼び出し
    save_collocations_to_csv(collocation_dict, output_file, output_pos, max_output)
    print(f"共起語分析の結果が {output_file} に保存されました。")
else:
    # 共起語が見つからなかった場合のメッセージをコンソールに表示
    print(f"'{keyword}'に対する共起語は見つかりませんでした。")
    print(f"検索モード: {search_type}, 品詞: {part_of_speech if part_of_speech else 'None'}")

# 例文検索機能
## ユーザーが設定するパラメーター

In [ ]:
lemma_to_extract = 'earth' # 抽出する単語を指定 
pos_to_extract = 'NOUN'  # 抽出する単語の品詞を指定
output_text_file = 'extract_file.txt'  # 抽出した文を保存するファイル名

In [ ]:
def extract_sentences_by_collocation(filtered_docs, collocation_dict, lemma, pos, output_file):
    """
    指定された共起語と品詞に対応する文を抽出し、テキストファイルに保存する。

    Args:
        filtered_docs (dict): フィルタリングされたDocオブジェクトの辞書
        collocation_dict (defaultdict): 共起語情報を格納する辞書(例 : {(lemma, pos): {'lemma': '', 'pos': '', 'freq': 0, 'indexes': []}, ...}
        lemma (str): 抽出する単語のレンマ
        pos (str): 抽出する単語の品詞
        output_file (str): 抽出した文を保存するファイル名
    """
    # 共起語と品詞に対応するインデックスを取得(例 : indexes = [(file_name, token_index), ...]
    indexes = collocation_dict.get((lemma, pos), {}).get('indexes', [])
    if not indexes:
        print(f"警告: 指定された共起語 ({lemma}, {pos}) に対応する例文が見つかりませんでした。")
        return
    
    # 処理された文を追跡するためのセット
    processed_sentences = set()

    # ドキュメントとテキスト情報のペアを用いて実際の文を抽出
    sentences = []
    for file_name, doc in filtered_docs.items():
        for token in doc:
            # トークンのインデックスと文書のテキスト情報を組み合わせたキーを作成
            token_key = (file_name, token.i)
            if token_key in indexes:
                sentence_text = token.sent.text
                # 重複を避けるために、すでに処理された文をチェック
                if sentence_text not in processed_sentences:
                    sentences.append(sentence_text)
                    processed_sentences.add(sentence_text)
    
    # テキストファイルに1行1文の形で出力
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            f.write(sentence + '\n')
    print(f"抽出された文({len(sentences)}件)が {output_file} に保存されました。")

# 文を抽出してテキストファイルに保存
extract_sentences_by_collocation(filtered_docs, collocation_dict, lemma_to_extract, pos_to_extract, output_text_file)

# **共起語分析（共起範囲指定）**
## ユーザーが設定するパラメーター

In [ ]:
# 基本設定
search_type = "phrase"
keyword = "hello world"  # 検索するキーワードを設定
range_left = 0    # キーワードの左側で検索する範囲を設定
range_right = 2   # キーワードの右側で検索する範囲を設定

# 追加設定
part_of_speech = "None"  # キーワードの品詞を設定

# 出力設定
output_pos = ["NOUN", "VERB", "ADJ", "PROPN", "ADV"]  # フィルターする品詞をリストで設定
max_output = 150 # 出力する行数を指定
output_file = "collocations_within_range.csv"  # 出力するCSVファイルの名前を設定

## 共起語分析の実行

In [ ]:
def search_word(doc, keyword, part_of_speech, collocation_dict, file_name, range_left, range_right):
    """
    キーワードの前後の共起範囲を指定して、単語を検索し、共起語を記録する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワード。
        part_of_speech (str): キーワードの品詞。Noneの場合は品詞を指定しない。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
        range_left (int): キーワードの左側で検索する範囲。
        range_right (int): キーワードの右側で検索する範囲。
    """
    for token in doc:
        if (part_of_speech is None or token.pos_ == part_of_speech) and token.text.lower() == keyword.lower():
            sentence = token.sent
            token_index_in_sentence = token.i - sentence.start
            relative_start = max(0, token_index_in_sentence - range_left)
            relative_end = min(len(sentence), token_index_in_sentence + range_right + 1)

            # キーワード自体は除外するが、範囲内の同じ単語は共起語としてカウント
            window_tokens = [
                sentence[i] for i in range(relative_start, relative_end)
                if i != token_index_in_sentence 
            ]

            record_collocations(window_tokens, collocation_dict, file_name)

def search_lemma(doc, keyword, part_of_speech, collocation_dict, file_name, range_left, range_right):
    """
    キーワードの前後の共起範囲を指定して、単語の原形を検索し、共起語を記録する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワードの原形。
        part_of_speech (str): キーワードの品詞。Noneの場合は品詞を指定しない。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
        range_left (int): キーワードの左側で検索する範囲。
        range_right (int): キーワードの右側で検索する範囲。
    """
    for token in doc:
        if (part_of_speech is None or token.pos_ == part_of_speech) and token.lemma_.lower() == keyword.lower():
            sentence = token.sent
            token_index_in_sentence = token.i - sentence.start
            relative_start = max(0, token_index_in_sentence - range_left)
            relative_end = min(len(sentence), token_index_in_sentence + range_right + 1)

            # キーワードのレンマ自体は除外するが、範囲内の同じレンマは共起語としてカウント
            window_tokens = [
                sentence[i] for i in range(relative_start, relative_end)
                if i != token_index_in_sentence
            ]

            record_collocations(window_tokens, collocation_dict, file_name)

def search_phrase(doc, phrase, collocation_dict, file_name, range_left, range_right):
    """
    キーワードの前後の共起範囲を指定して、フレーズを検索し、共起語を記録する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        phrase (str): 検索するフレーズ。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
        range_left (int): キーワードの左側で検索する範囲。
        range_right (int): キーワードの右側で検索する範囲。
    """
    # PhraseMatcherの初期化
    matcher = PhraseMatcher(doc.vocab, attr="LOWER")
    
    # フレーズのパターンをDocオブジェクトとして追加
    phrase_patterns = [nlp(phrase)]  # フレーズをDocオブジェクトに変換
    matcher.add("PHRASE", phrase_patterns)
    
    # ドキュメント内でフレーズを検索
    matches = matcher(doc)
    
    # マッチしたフレーズに対する処理
    for match_id, start, end in matches:
        span = doc[start:end]  # マッチしたフレーズのスパン
        sentence = span.sent  # スパンが含まれる文
        
        # マッチしたフレーズの開始トークンと終了トークンを基準にウィンドウサイズを適用
        window_start = max(span.start - range_left, sentence.start)
        window_end = min(span.end + range_right, sentence.end)
        
        # ウィンドウ内のトークンを抽出（フレーズ自体は除外）
        window_tokens = [sentence[i] for i in range(window_start, window_end) if i < span.start or i >= span.end]
        
        # フィルタリングされたトークンのリストを使用して共起語を記録
        record_collocations(window_tokens, collocation_dict, file_name)


def search_regex(doc, keyword, collocation_dict, file_name, range_left, range_right):
    """
    キーワードの前後の共起範囲を指定して、正規表現によるパターンを検索し、共起語を記録する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索する正規表現パターン。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
        range_left (int): キーワードの左側で検索する範囲。
        range_right (int): キーワードの右側で検索する範囲。
    """
    for sent in doc.sents:
        for match in re.finditer(keyword, sent.text):
            # キーワードにマッチした部分の文内インデックス範囲を取得
            start_idx, end_idx = match.start(), match.end()

            # キーワードにマッチしたトークンのインデックスを取得
            matched_token_indexes = [
                token.i for token in sent
                if token.idx >= start_idx + sent.start_char and token.idx < end_idx + sent.start_char
            ]

            # ウィンドウサイズに基づいてトークンを抽出
            window_start = max(matched_token_indexes[0] - range_left, sent.start)
            window_end = min(matched_token_indexes[-1] + range_right + 1, sent.end)

            # ウィンドウ内のトークンを抽出（キーワード自体は除外）
            window_tokens = [
                token for token in sent
                if token.i >= window_start and token.i < window_end and token.i not in matched_token_indexes
            ]

            record_collocations(window_tokens, collocation_dict, file_name)


def record_collocations(tokens, collocation_dict, file_name):
    """
    共起語の情報を辞書に記録する関数。

    Args:
        tokens (list): 共起語のトークンのリスト。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        file_name (str): 処理中のファイル名。
    """
    for token in tokens:
        key = (token.lemma_, token.pos_)
        collocation_dict[key]['lemma'] = token.lemma_
        collocation_dict[key]['pos'] = token.pos_
        collocation_dict[key]['freq'] += 1
        collocation_dict[key]['indexes'].append((file_name, token.i))

def perform_collocation_search(doc, keyword, search_type, collocation_dict, part_of_speech, file_name, range_left, range_right):
    """
    共起語分析を実行する関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワード。
        search_type (str): 検索モード（word, lemma, phrase, regex）。
        collocation_dict (defaultdict): 共起語を格納する辞書。
        part_of_speech (str): キーワードの品詞。Noneの場合は品詞を指定しない。
        file_name (str): 処理中のファイル名。
        range_left (int): キーワードの左側で検索する範囲。
        range_right (int): キーワードの右側で検索する範囲。

    Raises:
        ValueError: 無効な検索モードが指定された場合。
    """
    if search_type == 'word':
        search_word(doc, keyword, part_of_speech, collocation_dict, file_name, range_left, range_right)
    
    elif search_type == 'lemma':
        search_lemma(doc, keyword, part_of_speech, collocation_dict, file_name, range_left, range_right)
    
    elif search_type == 'phrase':
        search_phrase(doc, keyword, collocation_dict, file_name, range_left, range_right)

    elif search_type == 'regex':
        search_regex(doc, keyword, collocation_dict, file_name, range_left, range_right)


def save_collocations_to_csv(collocation_dict, output_file, output_pos=None, max_output=None):
    """
    共起語の情報をCSVファイルに保存する。

    Args:
        collocation_dict (defaultdict): 共起語情報を格納する辞書
        output_file (str): 出力ファイル名
        output_pos (list, optional): 出力する品詞のリスト。デフォルトはNone。
        max_output (int, optional): 出力する結果の最大数。デフォルトはNone。
    """
    # 辞書からデータフレームを作成
    df = pd.DataFrame.from_dict(collocation_dict, orient='index')
    
    # 全品詞に対するソート
    df_sorted_all = df.sort_values(by='freq', ascending=False)
    if max_output is not None:
        df_sorted_all = df_sorted_all.head(max_output)
    
    # 指定された品詞でフィルタリング
    dfs_filtered = []
    if output_pos:
        for pos in output_pos:
            df_filtered = df[df['pos'] == pos].sort_values(by='freq', ascending=False)
            if max_output is not None:
                df_filtered = df_filtered.head(max_output)
            df_filtered = df_filtered[['lemma', 'freq']].reset_index(drop=True)
            df_filtered.columns = [f'lemma_{pos}', f'freq_{pos}']
            dfs_filtered.append(df_filtered)
    
    # 指定された品詞に基づくデータフレームを結合
    df_filtered_combined = pd.concat(dfs_filtered, axis=1)
    # 全品詞の共起語データを追加
    df_final = pd.concat([df_filtered_combined, df_sorted_all[['lemma', 'pos', 'freq']].reset_index(drop=True)], axis=1)
    
    # CSVとして保存
    df_final.to_csv(output_file, index=False)


# 共起語を格納するための辞書
collocation_dict = defaultdict(lambda: {'lemma': '', 'pos': '', 'freq': 0, 'indexes': []})

#  品詞が指定されていない場合、検索対象の品詞をNoneに設定, search_typeがphraseまたはregexの場合もNoneに設定
if part_of_speech == "NONE" or part_of_speech == "" or search_type == 'phrase' or search_type == 'regex':
    part_of_speech = None

# 検索タイプが無効な場合はエラーを返す
if search_type not in ["word", "lemma", "regex", "phrase"]:
    raise ValueError("Invalid search type. Please choose from 'word', 'lemma', 'regex', or 'phrase'.")

# フィルタリングされたDocオブジェクトに対して共起語分析を実行
for file_name, doc in filtered_docs.items():
    perform_collocation_search(doc, keyword, search_type, collocation_dict, part_of_speech, file_name, range_left, range_right)

# 共起語の総数を計算し、結果を表示
total_matches = len(collocation_dict)
if total_matches > 0:
    # コンソールに共起語分析の結果概要を表示
    print(f"共起語分析の結果: {total_matches}件の共起語が見つかりました。")
    print(f"検索モード: {search_type}, キーワード: {keyword}, 品詞: {part_of_speech if part_of_speech else '指定なし'}")
    
    # 結果をCSVファイルに出力する関数を呼び出し
    save_collocations_to_csv(collocation_dict, output_file, output_pos, max_output)
    print(f"共起語分析の結果が {output_file} に保存されました。")
else:
    # 共起語が見つからなかった場合のメッセージをコンソールに表示
    print(f"'{keyword}'に対する共起語は見つかりませんでした。")
    print(f"検索モード: {search_type}, 品詞: {part_of_speech if part_of_speech else '指定なし'}")

# 例文出力機能
## ユーザーが設定するパラメーター

In [ ]:
lemma_to_extract = 'earth' # 抽出する単語を指定 
pos_to_extract = 'NOUN'  # 抽出する単語の品詞を指定
output_text_file = 'extract_file.txt'  # 抽出した文を保存するファイル名

In [ ]:
def extract_sentences_by_collocation(filtered_docs, collocation_dict, lemma, pos, output_file):
    """
    指定された共起語と品詞に対応する文を抽出し、テキストファイルに保存する。

    Args:
        filtered_docs (dict): フィルタリングされたDocオブジェクトの辞書
        collocation_dict (defaultdict): 共起語情報を格納する辞書(例 : {(lemma, pos): {'lemma': '', 'pos': '', 'freq': 0, 'indexes': []}, ...}
        lemma (str): 抽出する単語のレンマ
        pos (str): 抽出する単語の品詞
        output_file (str): 抽出した文を保存するファイル名
    """
    # 共起語と品詞に対応するインデックスを取得(例 : indexes = [(file_name, token_index), ...]
    indexes = collocation_dict.get((lemma, pos), {}).get('indexes', [])
    if not indexes:
        print(f"警告: 指定された共起語 ({lemma}, {pos}) に対応する例文が見つかりませんでした。")
        return
    
    # 処理された文を追跡するためのセット
    processed_sentences = set()

    # ドキュメントとテキスト情報のペアを用いて実際の文を抽出
    sentences = []
    for file_name, doc in filtered_docs.items():
        for token in doc:
            # トークンのインデックスと文書のテキスト情報を組み合わせたキーを作成
            token_key = (file_name, token.i)
            if token_key in indexes:
                sentence_text = token.sent.text
                # 重複を避けるために、すでに処理された文をチェック
                if sentence_text not in processed_sentences:
                    sentences.append(sentence_text)
                    processed_sentences.add(sentence_text)
    
    # テキストファイルに1行1文の形で出力
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            f.write(sentence + '\n')
    print(f"抽出された文({len(sentences)}件)が {output_file} に保存されました。")

# 文を抽出してテキストファイルに保存
extract_sentences_by_collocation(filtered_docs, collocation_dict, lemma_to_extract, pos_to_extract, output_text_file)